In [16]:
import os
import pandas as pd
from zipfile import ZipFile

from datetime import date, datetime, timedelta

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## Data Imports

In [2]:
os.chdir('/Volumes/USB30FD/LA-HEALTH-COVID-Data/data_10-02-2020/matched_data/')
RCAC_oct0120_df = pd.read_csv('rcac_poi_match.csv')

In [3]:
os.chdir('/Volumes/USB30FD/LA-HEALTH-COVID-Data/data_10-03-2020/matched_data/')
RCAC_oct0220_df = pd.read_csv('rcac_poi_match.csv')

### Import Safegraph POI

In [17]:
os.chdir('/Volumes/USB30FD/RMDI_data/')

# filename = 'core_places_us_only/2020/08/Core-USA-August2020-Release-CORE_POI-2020_07-2020-08-07.zip'
filename = 'core_places_us_only/09142020/Core-USA-Sep-CORE_POI-2020_08-2020-09-08.zip'

poi = []

with ZipFile(filename, 'r') as zip:
    with zip.open('core_poi-part1.csv.gz') as myfile:
        poi.append(pd.read_csv(myfile, compression='gzip'))
    with zip.open('core_poi-part2.csv.gz') as myfile:
        poi.append(pd.read_csv(myfile, compression='gzip'))
    with zip.open('core_poi-part3.csv.gz') as myfile:
        poi.append(pd.read_csv(myfile, compression='gzip'))
    with zip.open('core_poi-part4.csv.gz') as myfile:
        poi.append(pd.read_csv(myfile, compression='gzip'))
    with zip.open('core_poi-part5.csv.gz') as myfile:
        poi.append(pd.read_csv(myfile, compression='gzip'))

poi_df = pd.concat(poi)

del poi

POI_DF = poi_df[poi_df['region']=='CA']

del poi_df

In [18]:
POI_DF.head(2)

,safegraph_place_id,parent_safegraph_place_id,location_name,safegraph_brand_ids,brands,top_category,sub_category,naics_code,latitude,longitude,street_address,city,region,postal_code,iso_country_code,phone_number,open_hours,category_tags
6,sg:00558d9ab88d4ad9998e3bb43c3f6f60,NaN,The Insurance Place,NaN,NaN,"Agencies, Brokerages, and Other Insurance Rela...",Insurance Agencies and Brokerages,524210.0,33.124147,-117.275411,2111 Palomar Airport Rd Ste 322,Carlsbad,CA,92011,US,1.760643e+10,"{ ""Mon"": [[""9:00"", ""17:30""]], ""Tue"": [[""9:00"",...",NaN
8,sg:008eaaf6ef3a43bea9757af6a79ff239,NaN,Wells Fargo,SG_BRAND_d8326a9cc6413b12d8df3c617490e882,Wells Fargo,Depository Credit Intermediation,Commercial Banking,522110.0,37.736914,-122.198516,433 Hegenberger Rd Ste 110,Oakland,CA,94621,US,1.510383e+10,"{ ""Mon"": [[""9:00"", ""18:00""]], ""Tue"": [[""9:00"",...",NaN


## Exploring imported data

In [4]:
RCAC_oct0120_df.shape

(467, 13)

In [5]:
RCAC_oct0120_df.sort_values('number_of_candidate_matches', ascending=False)

,customer_location_name,customer_city_name,customer_number_of_confirmed_staff,customer_number_of_confirmed_residents,customer_total_deaths,customer_city,customer_state,customer_postal_code,other_match_candidates,number_of_candidate_matches,warnings,is_closed,safegraph_place_id
377,The Village at Sherman Oaks,"Sherman Oaks, CA",1,3,0,Sherman Oaks,CA,91403,"sg:55bc747692a8405194883766beac2c6b,sg:ce3c236...",8,MISSING_STREET_ADDRESS,False,sg:1dfed36a59ff4b59b23c3a62eae9198e
376,The Village at Sherman Oaks,"Sherman Oaks, CA",1,3,0,Sherman Oaks,CA,91403,"sg:55bc747692a8405194883766beac2c6b,sg:ce3c236...",8,MISSING_STREET_ADDRESS,False,sg:1dfed36a59ff4b59b23c3a62eae9198e
64,Brookdale Assisted Living North Tarzana RCFE,"Tarzana, CA",6,4,1,Tarzana,CA,91356,"sg:a787de65f2694e06baa9c402396aee1a,sg:0e11001...",4,MISSING_STREET_ADDRESS,False,sg:4baf4c41136045b2945572b6196425b4
66,Brookdale Assisted Living North Tarzana RCFE,"Tarzana, CA",6,4,1,Tarzana,CA,91356,"sg:a787de65f2694e06baa9c402396aee1a,sg:0e11001...",4,MISSING_STREET_ADDRESS,False,sg:4baf4c41136045b2945572b6196425b4
228,Kei-Ai Los Angeles Healthcare Center,"Los Angeles, CA",37,25,8,Los Angeles,CA,90045,"sg:75838b09fe99470080be9a534f004098,sg:8f3f540...",3,MISSING_STREET_ADDRESS,False,sg:f5ad31b930034c858e251f10fbfbbcb5
346,Sherman Oaks Health & Rehabilitation Center,"Sherman Oaks, CA",28,54,13,Sherman Oaks,CA,91403,"sg:736d114309144697af0887faee8648c1,sg:e02440d...",3,MISSING_STREET_ADDRESS,False,sg:55bc747692a8405194883766beac2c6b
345,Sherman Oaks Health & Rehabilitation Center,"Sherman Oaks, CA",28,54,13,Sherman Oaks,CA,91403,"sg:736d114309144697af0887faee8648c1,sg:e02440d...",3,MISSING_STREET_ADDRESS,False,sg:55bc747692a8405194883766beac2c6b
226,Jackman Street House (Tarzana Treatment Center),"Lancaster, CA",0,3,0,Lancaster,CA,93534,"sg:4e56f5f6fed5458db22e93b33256e493,sg:f14b9c3...",3,MISSING_STREET_ADDRESS,False,sg:d67d278f939b4c2a8f199035abe60bc3
172,Glendora Grand Convalescent Hospital,"Glendora, CA",91,240,16,Glendora,CA,91740,sg:f1295a9bd60947f8b2c52f85d2320171,2,MISSING_STREET_ADDRESS,False,sg:70da55eeae3244e3a91b5f047ff650e1
251,Motion Picture & Television Fund,"Woodland Hills, CA",16,17,7,Woodland Hills,CA,91302,sg:4c7434e9d398436a9581eea35372fac3,2,MISSING_STREET_ADDRESS,False,sg:c13b2437a37941c4a6cc7cd4939011df


In [6]:
## 467 rows
len(RCAC_oct0120_df['customer_location_name'])

467

In [7]:
## 349 have 1 match
len(RCAC_oct0120_df[RCAC_oct0120_df['number_of_candidate_matches']==1]['customer_location_name']) 


349

In [8]:
## 18 have > 1 match
len(RCAC_oct0120_df[RCAC_oct0120_df['number_of_candidate_matches']>1]['customer_location_name']) 

18

In [9]:
## 100 POI are not found in safegraph
len(RCAC_oct0120_df[RCAC_oct0120_df['number_of_candidate_matches']==0]['customer_location_name']) 


100

### Get unique count of POI names

In [13]:
len(RCAC_oct0120_df[RCAC_oct0120_df['number_of_candidate_matches']==1]['customer_location_name'].unique())


(205, 12)

### Get unique count of POI id

In [23]:
len(RCAC_oct0120_df[RCAC_oct0120_df['number_of_candidate_matches']==1]['safegraph_place_id'].unique())


In [77]:
# RCAC_oct0120_df[RCAC_oct0120_df['number_of_candidate_matches']==1].groupby('safegraph_place_id').count()


### Investigate shared data between imported matched and SAFEGRAPH POI

In [21]:
# POI_DF.shape # 704316, 18
POI_DF.head(2)

,safegraph_place_id,parent_safegraph_place_id,location_name,safegraph_brand_ids,brands,top_category,sub_category,naics_code,latitude,longitude,street_address,city,region,postal_code,iso_country_code,phone_number,open_hours,category_tags
6,sg:00558d9ab88d4ad9998e3bb43c3f6f60,NaN,The Insurance Place,NaN,NaN,"Agencies, Brokerages, and Other Insurance Rela...",Insurance Agencies and Brokerages,524210.0,33.124147,-117.275411,2111 Palomar Airport Rd Ste 322,Carlsbad,CA,92011,US,1.760643e+10,"{ ""Mon"": [[""9:00"", ""17:30""]], ""Tue"": [[""9:00"",...",NaN
8,sg:008eaaf6ef3a43bea9757af6a79ff239,NaN,Wells Fargo,SG_BRAND_d8326a9cc6413b12d8df3c617490e882,Wells Fargo,Depository Credit Intermediation,Commercial Banking,522110.0,37.736914,-122.198516,433 Hegenberger Rd Ste 110,Oakland,CA,94621,US,1.510383e+10,"{ ""Mon"": [[""9:00"", ""18:00""]], ""Tue"": [[""9:00"",...",NaN


In [22]:
# RCAC_oct0120_df.shape # 467, 13
RCAC_oct0120_df.head(2)

,customer_location_name,customer_city_name,customer_number_of_confirmed_staff,customer_number_of_confirmed_residents,customer_total_deaths,customer_city,customer_state,customer_postal_code,other_match_candidates,number_of_candidate_matches,warnings,is_closed,safegraph_place_id
0,A and M Armour Home Care,"Pomona, CA",7,4,0,Pomona,CA,91767,NaN,0,MISSING_STREET_ADDRESS,False,NaN
1,A and M Armour Home Care,"Pomona, CA",7,4,0,Pomona,CA,91769,NaN,0,MISSING_STREET_ADDRESS,False,NaN


In [36]:
single_matches = RCAC_oct0120_df[RCAC_oct0120_df['number_of_candidate_matches']==1]
single_matches.shape # (349, 13)
single_matches.head()

,customer_location_name,customer_city_name,customer_number_of_confirmed_staff,customer_number_of_confirmed_residents,customer_total_deaths,customer_city,customer_state,customer_postal_code,other_match_candidates,number_of_candidate_matches,warnings,is_closed,safegraph_place_id
6,Affinity Healthcare Center,"Paramount, CA",6,6,0,Paramount,CA,90723,NaN,1,MISSING_STREET_ADDRESS,False,sg:6df8b697b0374c44a5b5b72dab2ec481
7,Alameda Care Center,"Burbank, CA",2,4,0,Burbank,CA,91501,NaN,1,MISSING_STREET_ADDRESS,False,sg:7c8fd2469df44ba1968097a4156aa3da
8,Alameda Care Center,"Burbank, CA",2,4,0,Burbank,CA,91503,NaN,1,MISSING_STREET_ADDRESS,False,sg:7c8fd2469df44ba1968097a4156aa3da
9,Alcott Rehabilitation Hospital,"Los Angeles, CA",10,22,3,Los Angeles,CA,90045,NaN,1,MISSING_STREET_ADDRESS,False,sg:33bffe14ad4d484ba9434bf1e6ac939c
10,"Alhambra Healthcare & Wellness Centre, LP","Alhambra, CA",1,3,0,Alhambra,CA,91801,NaN,1,MISSING_STREET_ADDRESS,False,sg:c8d4381aa6da48a88a59b82ec389a5bd


In [40]:
pd.merge(single_matches, POI_DF, on=['safegraph_place_id'], how='left')


,customer_location_name,customer_city_name,customer_number_of_confirmed_staff,customer_number_of_confirmed_residents,customer_total_deaths,customer_city,customer_state,customer_postal_code,other_match_candidates,number_of_candidate_matches,warnings,is_closed,safegraph_place_id,parent_safegraph_place_id,location_name,safegraph_brand_ids,brands,top_category,sub_category,naics_code,latitude,longitude,street_address,city,region,postal_code,iso_country_code,phone_number,open_hours,category_tags
0,Affinity Healthcare Center,"Paramount, CA",6,6,0,Paramount,CA,90723,NaN,1,MISSING_STREET_ADDRESS,False,sg:6df8b697b0374c44a5b5b72dab2ec481,NaN,Affinity Healthcare Center,NaN,NaN,Nursing Care Facilities (Skilled Nursing Facil...,Nursing Care Facilities (Skilled Nursing Facil...,623110.0,33.889519,-118.175942,7039 Alondra Blvd,Paramount,CA,90723.0,US,1.562531e+10,"{ ""Mon"": [[""0:00"", ""5:30""], [""6:00"", ""24:00""]]...",NaN
1,Alameda Care Center,"Burbank, CA",2,4,0,Burbank,CA,91501,NaN,1,MISSING_STREET_ADDRESS,False,sg:7c8fd2469df44ba1968097a4156aa3da,NaN,Alameda Care Center,NaN,NaN,Nursing Care Facilities (Skilled Nursing Facil...,Nursing Care Facilities (Skilled Nursing Facil...,623110.0,34.163811,-118.313025,925 W Alameda Ave,Burbank,CA,91506.0,US,1.818843e+10,NaN,NaN
2,Alameda Care Center,"Burbank, CA",2,4,0,Burbank,CA,91503,NaN,1,MISSING_STREET_ADDRESS,False,sg:7c8fd2469df44ba1968097a4156aa3da,NaN,Alameda Care Center,NaN,NaN,Nursing Care Facilities (Skilled Nursing Facil...,Nursing Care Facilities (Skilled Nursing Facil...,623110.0,34.163811,-118.313025,925 W Alameda Ave,Burbank,CA,91506.0,US,1.818843e+10,NaN,NaN
3,Alcott Rehabilitation Hospital,"Los Angeles, CA",10,22,3,Los Angeles,CA,90045,NaN,1,MISSING_STREET_ADDRESS,False,sg:33bffe14ad4d484ba9434bf1e6ac939c,NaN,Alcott Rehabilitation Hospital,NaN,NaN,Nursing Care Facilities (Skilled Nursing Facil...,Nursing Care Facilities (Skilled Nursing Facil...,623110.0,34.052867,-118.317157,3551 W Olympic Blvd,Los Angeles,CA,90019.0,US,1.323737e+10,"{ ""Mon"": [[""8:00"", ""18:00""]], ""Tue"": [[""8:00"",...",NaN
4,"Alhambra Healthcare & Wellness Centre, LP","Alhambra, CA",1,3,0,Alhambra,CA,91801,NaN,1,MISSING_STREET_ADDRESS,False,sg:c8d4381aa6da48a88a59b82ec389a5bd,NaN,Alhambra Healthcare & Wellness Centre Lp,NaN,NaN,Nursing Care Facilities (Skilled Nursing Facil...,Nursing Care Facilities (Skilled Nursing Facil...,623110.0,34.090421,-118.124228,415 S Garfield Ave,Alhambra,CA,91801.0,US,1.626282e+10,NaN,NaN
5,"Alhambra Healthcare & Wellness Centre, LP","Alhambra, CA",1,3,0,Alhambra,CA,91804,NaN,1,MISSING_STREET_ADDRESS,False,sg:c8d4381aa6da48a88a59b82ec389a5bd,NaN,Alhambra Healthcare & Wellness Centre Lp,NaN,NaN,Nursing Care Facilities (Skilled Nursing Facil...,Nursing Care Facilities (Skilled Nursing Facil...,623110.0,34.090421,-118.124228,415 S Garfield Ave,Alhambra,CA,91801.0,US,1.626282e+10,NaN,NaN
6,"Alhambra Healthcare & Wellness Centre, LP","Alhambra, CA",1,3,0,Alhambra,CA,91802,NaN,1,MISSING_STREET_ADDRESS,False,sg:c8d4381aa6da48a88a59b82ec389a5bd,NaN,Alhambra Healthcare & Wellness Centre Lp,NaN,NaN,Nursing Care Facilities (Skilled Nursing Facil...,Nursing Care Facilities (Skilled Nursing Facil...,623110.0,34.090421,-118.124228,415 S Garfield Ave,Alhambra,CA,91801.0,US,1.626282e+10,NaN,NaN
7,Angels Nursing Health Center,"Los Angeles, CA",4,1,0,Los Angeles,CA,90045,NaN,1,MISSING_STREET_ADDRESS,False,sg:8a56316a43dd437095fe8237c6522d50,NaN,Angels Nursing Health Center,NaN,NaN,Nursing Care Facilities (Skilled Nursing Facil...,Nursing Care Facilities (Skilled Nursing Facil...,623110.0,34.059515,-118.268887,415 S Union Ave,Los Angeles,CA,90017.0,US,1.213484e+10,NaN,NaN
8,Antelope Valley Care Center,"Lancaster, CA",32,27,3,Lancaster,CA,93534,NaN,1,MISSING_STREET_ADDRESS,False,sg:268d071ec4024118a084c4b98dd3c616,NaN,Antelope Valley Care Center,NaN,NaN,Nursing Care Facilities (Skilled Nursing Facil...,Nursing Care Facilities (Skilled Nursing Facil...,623110.0,34.692332,-118.157127,44567 15t

## Explore mergings

In [35]:
oct1_oct2_df = pd.merge(RCAC_oct0120_df, RCAC_oct0220_df, on=['location_name', 'city_name'], how='inner')


In [57]:
RCAC_oct0220_df[RCAC_oct0220_df['location_name']=='Elwyn El Monte']

,location_name,city_name,number_of_confirmed_staff,number_of_confirmed_residents,total_deaths
62,Elwyn El Monte,"El Monte, CA",8,1,0


In [58]:
RCAC_oct0120_df[RCAC_oct0120_df['location_name']=='Elwyn El Monte']

,location_name,city_name,number_of_confirmed_staff,number_of_confirmed_residents,total_deaths
65,Elwyn El Monte,"El Monte, CA",6,1,0


In [111]:
oct1_oct2_df.head(2)

,location_name,city_name,number_of_confirmed_staff_x,number_of_confirmed_residents_x,total_deaths_x,number_of_confirmed_staff_y,number_of_confirmed_residents_y,total_deaths_y
0,A and M Armour Home Care,"Pomona, CA",7,4,0,7,4,0
1,Affinity Healthcare Center,"Paramount, CA",6,6,0,6,6,0


In [82]:
today = date.today()
today = today - timedelta(days=1)
today = today.strftime("%m-%d-%Y")

In [83]:
print(today)

10-02-2020


In [109]:
oct2_join_df = pd.DataFrame(columns=['date', 'location_name', 'city', 'confirmed_staff_delta', 'confirmed_residents_delta', 'total_deaths_delta'])

oct2_join_df['location_name'] = oct1_oct2_df['location_name']
oct2_join_df['city'] = oct1_oct2_df['city_name']
oct2_join_df['confirmed_staff_delta'] = oct1_oct2_df['number_of_confirmed_staff_y'] - oct1_oct2_df['number_of_confirmed_staff_x']
oct2_join_df['confirmed_residents_delta'] = oct1_oct2_df['number_of_confirmed_residents_y'] - oct1_oct2_df['number_of_confirmed_residents_x']
oct2_join_df['total_deaths_delta'] = oct1_oct2_df['total_deaths_y'] - oct1_oct2_df['total_deaths_x']
oct2_join_df['date'] = today
oct2_join_df.sort_values('confirmed_staff_delta', ascending=False)


,date,location_name,city,confirmed_staff_delta,confirmed_residents_delta,total_deaths_delta
186,10-02-2020,Windsor Gardens Convalescent Hospital,"Los Angeles, CA",2,1,2
61,10-02-2020,Elwyn El Monte,"El Monte, CA",2,0,0
143,10-02-2020,Sherman Oaks Health & Rehabilitation Center,"Sherman Oaks, CA",1,0,0
70,10-02-2020,Golden Legacy (Previously San Fernando Post Ac...,"Sylmar, CA",1,0,0
31,10-02-2020,Buena Ventura Post Acute Care Center,"Los Angeles, CA",1,0,0
20,10-02-2020,Beacon Healthcare Center,"West Covina, CA",1,1,0
159,10-02-2020,Topanga Terrace Convalescent Center,"West Hills, CA",1,1,0
43,10-02-2020,Claremont Care Center,"Pomona, CA",1,0,0
131,10-02-2020,Rinaldi Convalescent Hospital,"Granada Hills, CA",0,0,0
125,10-02-2020,Pathways Sober Living,"Pomona, CA",0,0,0


In [127]:
RCAC_oct0120_df.head() # (205, 5)

,location_name,city_name,number_of_confirmed_staff,number_of_confirmed_residents,total_deaths
0,A and M Armour Home Care,"Pomona, CA",7,4,0
1,AAA Royal Senior Living,"Tarzana, CA",0,1,1
2,Affinity Healthcare Center,"Paramount, CA",6,6,0
3,Alameda Care Center,"Burbank, CA",2,4,0
4,Alcott Rehabilitation Hospital,"Los Angeles, CA",10,22,3


In [132]:
RCAC_oct0220_df.head() # (197, 5)

,location_name,city_name,number_of_confirmed_staff,number_of_confirmed_residents,total_deaths
0,A and M Armour Home Care,"Pomona, CA",7,4,0
1,Affinity Healthcare Center,"Paramount, CA",6,6,0
2,Alameda Care Center,"Burbank, CA",2,4,0
3,Alcott Rehabilitation Hospital,"Los Angeles, CA",10,22,3
4,"Alhambra Healthcare & Wellness Centre, LP","Alhambra, CA",1,3,0


In [129]:
oct1_oct2_df = pd.merge(RCAC_oct0120_df, RCAC_oct0220_df, on=['location_name', 'city_name'], how='inner')

RCAC_oct0220_df[(~RCAC_oct0220_df['location_name'].isin(oct1_oct2_df['location_name']))&(~RCAC_oct0220_df['city_name'].isin(oct1_oct2_df['city_name']))]



,location_name,city_name,number_of_confirmed_staff,number_of_confirmed_residents,total_deaths
31,Brookfield Healthcare Center,"Downey, CA",6,18,6
109,Mother Gertrude Home Assisted Living,"San Fernando, CA",2,10,1


In [124]:
oct2_new_df = pd.DataFrame(columns=['date', 'location_name', 'city', 'confirmed_staff_delta', 'confirmed_residents_delta', 'total_deaths_delta'])

oct_2_new_df.shape
oct2_new_df['location_name'] = oct1_oct2_df['location_name']
oct2_new_df['city'] = oct1_oct2_df['city_name']
oct2_new_df['confirmed_staff_delta'] = oct1_oct2_df['number_of_confirmed_staff_y'] - oct1_oct2_df['number_of_confirmed_staff_x']
oct2_new_df['confirmed_residents_delta'] = oct1_oct2_df['number_of_confirmed_residents_y'] - oct1_oct2_df['number_of_confirmed_residents_x']
oct2_new_df['total_deaths_delta'] = oct1_oct2_df['total_deaths_y'] - oct1_oct2_df['total_deaths_x']
oct2_new_df['date'] = today
oct2_new_df.sort_values('confirmed_staff_delta', ascending=False)
